<a href="https://colab.research.google.com/github/lrssv/ComputacaoEvolutiva/blob/Part-2/MutationandCrossover.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model for classifying the Perpetrator's gender using an evolutionary algorithm to select features.

## - Mutation and Crossover

This code has all steps for the Mutation and Crossover. Where:

* Imports: the necessary libraries and the population; 
* Scripts: steps for do mutation or crossover;
* Setups: results.


## Imports

In [1]:
import pickle
import pandas as pd
import numpy as np
import random
import math

In [2]:
with open("/content/drive/My Drive/Data Files/fitness.txt", "rb") as fp:   
  generation = pickle.load(fp)

## Scripts

In [180]:
def reproduction(population,tau):
  children = []
  probs = []
  pop = [i[1] for i in population]
  
  for p in population:
    rand = random.uniform(0,1) #random value to compare with prob_mutation
    parent1 = random.choice(pop) 

    cp1 = parent1[1][1] #crossover_probability parent 1
    ms1 = parent1[1][0] #mutation_strength parent 1 
    
    parent2 = random.choice(pop)
    cp2 = parent2[1][1] #crossover_probability parent 2
    ms2 = parent2[1][0] #mutation_strength parent 2

    crossover_probability = (cp1 + cp2)/2 
    parent1_features = [i==1 for i in parent1[2:]]
    parent2_features = [i==1 for i in parent2[2:]]
    
    if rand < crossover_probability:
      #crossover
      n = random.uniform(0,1)
      chromosome_length = len(parent1_features) #length of chromosome
      crossover_point = random.randint(1,chromosome_length-1) #point of crossover
      
      child1 = parent1_features[1:crossover_point] + parent2_features[crossover_point:]
      child2 = parent2_features[1:crossover_point] + parent1_features[crossover_point:]

      child_mutation_strength = n*ms1 + (1-n)*ms2
      child_crossover_probability = n*cp1 + (1-n)*cp2
      
      probs.append(child_mutation_strength)
      probs.append(child_crossover_probability)

      child1.insert(0,parent2[0]) #model+child1
      child2.insert(0,parent1[0]) #model+child2
      child1.insert(1,probs)
      child2.insert(1,probs)
      
      children.append(child1)
      children.append(child2)

      probs = []
    
    else:
      #mutation    
      n = random.uniform(0,1)

      mutation_strength = ms1 * math.exp(tau*n)  
      crossover_probability = cp1 * math.exp(tau*n)

      mutation_strength = reflect(mutation_strength,0.001,0.999)
      crossover_probability = reflect(crossover_probability,0.001,0.999)

      probs.append(mutation_strength)
      probs.append(crossover_probability)

      if n < 0.5:
         parent1[0] = random.choice(range(4)) #mute the model
      
      for i in range(len(parent1_features)): 
        if n < 0.5:
          parent1_features[i] = not parent1_features[i]
      
      child = parent1_features
      child.insert(0,parent1[0])
      child.insert(1,probs)
      children.append(child)

      probs = []
  
  return children      

In [31]:
def reflect(v,lb,ub):
  while (v < lb or v > ub):
    if (v < lb):
      v += 2*(lb - v);
    if (v > ub):
      v -= 2*(v - ub);
  
  return v;

## Setups

In [153]:
tau = 0.5

In [184]:
run = reproduction(generation,tau)

In [185]:
pd.DataFrame(run)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0,"[0.058823529411764705, 0.8]",True,True,True,True,True,True,False,True,True,False,False,False,True,False,True,False,None
1,0,"[0.058823529411764705, 0.8]",False,True,False,False,True,False,True,True,True,False,True,False,False,False,True,True,None
2,1,"[0.08948061690358393, 0.7810636101112585]",False,False,True,False,False,True,True,True,False,False,False,True,False,False,True,True,True
3,2,"[0.058823529411764705, 0.8]",False,False,True,True,False,True,False,True,False,False,False,True,True,True,False,False,None
4,0,"[0.058823529411764705, 0.8]",True,True,False,True,False,False,True,False,False,False,False,False,False,True,True,True,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,2,"[0.058823529411764705, 0.8]",True,True,False,True,False,True,False,False,False,False,True,False,True,True,True,False,None
172,2,"[0.058823529411764705, 0.8]",True,False,False,False,False,True,False,False,True,True,False,True,False,True,False,False,None
173,3,"[0.058823529411764705, 0.8]",False,False,True,True,True,True,False,False,True,False,False,False,True,True,False,False,None
174,3,"[0.058823529411764705, 0.8]",True,True,True,False,True,True,True,False,True,False,False,True,True,True,False,False,None


In [ ]:
with open("/content/drive/My Drive/Data Files/mutandcross.txt", "wb") as mec: 
  pickle.dump(run, mec)